In [15]:
import requests
import json
import datetime



In [16]:
# start, end, step

# def get_bounds():
now = datetime.datetime.now()
start = (now - datetime.timedelta(minutes=10)).timestamp()
end = now.timestamp()
# step = datetime.timedelta(seconds=5)


In [30]:
url = (
    f'http://127.0.0.1:9090/api/v1/query_range'
    f'?query=rate(example_app_test_metric_sum[1m])&start={start}&end={end}&step=5'
)
response = requests.get(url)
response = json.loads(response.text)
values = [(float(v[1]), v[0]) for v in response['data']['result'][0]['values']]

In [31]:
len(values)

121

In [32]:
values

[(7624.405888180066, 1575020060.369),
 (7674.184558359239, 1575020065.369),
 (7715.461142511098, 1575020070.369),
 (7731.12566787346, 1575020075.369),
 (7737.811443471427, 1575020080.369),
 (7714.591686945559, 1575020085.369),
 (7714.633096975, 1575020090.369),
 (7882.874898290811, 1575020095.369),
 (7822.1544971026005, 1575020100.369),
 (7792.745461901769, 1575020105.369),
 (7790.257878500249, 1575020110.369),
 (7776.12200184879, 1575020115.369),
 (7763.976238371376, 1575020120.369),
 (7749.773153923282, 1575020125.369),
 (7708.194197730666, 1575020130.369),
 (7680.978469922191, 1575020135.369),
 (7664.605168743876, 1575020140.369),
 (7673.7666194810845, 1575020145.369),
 (7662.798416704782, 1575020150.369),
 (7697.115261961052, 1575020155.369),
 (7723.051300010325, 1575020160.369),
 (7752.7705800767235, 1575020165.369),
 (7762.626180766615, 1575020170.369),
 (7773.963130665714, 1575020175.369),
 (7775.605915231079, 1575020180.369),
 (7829.5643637620215, 1575020185.369),
 (7857.813620

In [36]:
import datetime
from typing import Any, List, Union

import pandas as pd
# from fbprophet import Prophet

from storage import Storage, GraphiteStorage, PrometheusStorage


class TaskRunner:
    def __init__(self, storage, predictor):
        self.storage = storage
        self.predictor = predictor

    def run_task(self, metric_name: str, need_plot: bool = False):
        minutes_to_get_metrics = 10
        current_timestamp = datetime.datetime.now()
        from_timestamp = current_timestamp
        from_timestamp -= datetime.timedelta(
            minutes=minutes_to_get_metrics,
        )

        metric_values = self.storage.read_metrics(
            metric_name=metric_name,
            from_timestamp=from_timestamp,
        )

        metrics_as_pddf = pd.DataFrame(
            data=[
                (datetime.datetime.fromtimestamp(ts), value) 
                for value,ts in metric_values 
                if value is not None
            ],
            columns=['ds', 'y'],
        )
        metrics_as_pddf['ds'] = pd.to_datetime(metrics_as_pddf['ds'])

        train_test_ts_bound = from_timestamp
        train_test_bs_bound += datetime.timedelta(
            minutes=int(minutes_to_get_metrics * 0.9),
        )

        df_train, df_test = (
            metrics_as_pddf[metrics_as_pddf['ds'] <= train_test_ts_bound],
            metrics_as_pddf[metrics_as_pddf['ds'] > train_test_ts_bound],
        )

        self.predictor.fit(df_train)

        future_predictions = self.predictor.predict(df_test[['ds']])

        df_test['yhat_lower'] = future_predictions['yhat_lower']
        df_test['yhat'] = future_predictions['yhat']
        df_test['yhat_upper'] = future_predictions['yhat_upper']

        bad_points = []
        for ds, y, yhat_lower, yhat, yhat_upper in df_tez.values:
            if not (yhat_lower <= y <= yhat_upper):
                bad_points.append((
                    ds,
                    y,
                    yhat_lower,
                    yhat,
                    yhat_upper,
                ))

        if need_plot:
            self.predictor.plot(future_predictions)


        return metric_name, bad_points

ImportError: cannot import name 'PrometheusStorage'